In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics
from utils.watchhouse import DirectLimitWatchman, PcaLimitWatchman
from utils.watchhouse import IsoForestWatchman
from utils.watchhouse import LinearPredictWatchman, DeepPredictWatchman

In [4]:
SEED = 127

# Preparing datasets

In [5]:
datasets = {
    0: GhlKasperskyDataset(),
    1: TepHarvardDataset(),
    2: TepKasperskyDataset(),
    3: SwatItrustDataset(),
}

In [6]:
shake_kwargs = {
    'random_state': SEED,
    'valid2test_ratio': 0.3,
}
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)

# Preparing watchhouse

In [7]:
watchhouse = {
    0: {  # GhlKasperskyDataset
        0: DirectLimitWatchman(),
        1: PcaLimitWatchman(),
        2: IsoForestWatchman(),
        3: LinearPredictWatchman(),
        4: DeepPredictWatchman(),
    },
    1: {  # TepHarvardDataset
        0: DirectLimitWatchman(),
        1: PcaLimitWatchman(),
        2: IsoForestWatchman(),
        3: LinearPredictWatchman(),
        4: DeepPredictWatchman(),
    },
    2: {  # TepKasperskyDataset
        0: DirectLimitWatchman(),
        1: PcaLimitWatchman(),
        2: IsoForestWatchman(),
        3: LinearPredictWatchman(),
        4: DeepPredictWatchman(),
    },
    3: {  # SwatItrustDataset
        0: DirectLimitWatchman(),
        1: PcaLimitWatchman(),
        2: IsoForestWatchman(),
        3: LinearPredictWatchman(),
        4: DeepPredictWatchman(),
    },
}

In [8]:
for d in datasets:
    for w in watchhouse[d]:
        watchhouse[d][w].load(f'{datasets[d].__class__.__name__}')

# Examine

## Throw stones

In [10]:
stones = dict()
for d in datasets:
    stones[d] = dict()
    for w in watchhouse[d]:
        stones[d][w] = list()

In [11]:
for d in datasets:
    for data, faults, info in tqdm(datasets[d].test_generator(), desc=f'Detect on test {d}'):
        for w in watchhouse[d]:
            detect = watchhouse[d][w].predict(data)
            stones[d][w].append(pd.concat([faults, detect], axis=1))
            stones[d][w][-1].index.name = info

Detect on test 0:   0%|          | 0/48 [00:00<?, ?it/s]

Detect on test 1:   0%|          | 0/10500 [00:00<?, ?it/s]

Detect on test 2:   0%|          | 0/488 [00:00<?, ?it/s]

Detect on test 3:   0%|          | 0/6 [00:00<?, ?it/s]

## Results

In [12]:
metrics = ('precision', 'recall', 'f1_score')
results = {str(datasets[d]): pd.DataFrame(columns=metrics) for d in datasets}

### Individual

In [13]:
for d in datasets:
    for w in tqdm(watchhouse[d], desc=f'Collect stones d{d}'):
        exam_paper = pd.DataFrame(columns=metrics)
        for i_st, st in enumerate(stones[d][w]):
            exam_paper.loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], st.iloc[:, 1:])
        results[str(datasets[d])].loc[str(watchhouse[d][w]), metrics] = exam_paper.mean().values

Collect stones d0:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d1:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d2:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d3:   0%|          | 0/5 [00:00<?, ?it/s]

### Ensembling

In [18]:
threshold = 5
for d in tqdm(datasets):
    exam_paper = pd.DataFrame(columns=metrics)
    for i_st, st in enumerate(stones[d][0]):
        opinions = pd.concat([stones[d][w][i_st].iloc[:, 1:].sum(axis=1) for w in watchhouse[d]], axis=1)
        detect = (opinions.sum(axis=1) >= threshold).astype('uint8')
        exam_paper.loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], detect)
    results[str(datasets[d])].loc[f'ensemble(threshold={threshold})', metrics] = exam_paper.mean().values

  0%|          | 0/4 [00:00<?, ?it/s]

### Let's see

In [19]:
for d in results:
    print(d)
    display(results[d].sort_values(by='f1_score', ascending=False))

GhlKasperskyDataset(E:\Datasets\GHL)


,precision,recall,f1_score
ensemble(threshold=5),0.738343,0.532937,0.593724
ensemble(threshold=4),0.493247,0.602728,0.494965
ensemble(threshold=3),0.334795,0.66002,0.401773
PcaLimitWatchman(n_features=12),0.5075,0.377034,0.388211
DirectLimitWatchman(n_features=12),0.32591,0.654315,0.379217
DeepPredictWatchman(n_features=12),0.222512,0.710565,0.322436
IsoForestWatchman(n_features=12),0.026413,0.832391,0.050855
LinearPredictWatchman(n_features=12),0.027397,0.042014,0.028886


TepHarvardDataset(E:\Datasets\TEP\dataverse)


,precision,recall,f1_score
LinearPredictWatchman(n_features=52),0.980548,0.798,0.812021
ensemble(threshold=3),0.9965,0.727969,0.761344
ensemble(threshold=4),0.998278,0.689536,0.727105
DeepPredictWatchman(n_features=52),0.984308,0.678536,0.716741
DirectLimitWatchman(n_features=52),0.986124,0.644344,0.682156
ensemble(threshold=5),0.999062,0.622593,0.674236
PcaLimitWatchman(n_features=52),0.996583,0.558771,0.600721
IsoForestWatchman(n_features=52),0.853757,0.478014,0.525161


TepKasperskyDataset(E:\Datasets\TEP\kaspersky)


,precision,recall,f1_score
LinearPredictWatchman(n_features=53),0.784132,0.480178,0.443834
ensemble(threshold=3),0.749968,0.470508,0.427041
DirectLimitWatchman(n_features=53),0.776658,0.431569,0.415414
ensemble(threshold=4),0.770535,0.438411,0.410373
DeepPredictWatchman(n_features=53),0.651948,0.480615,0.391748
ensemble(threshold=5),0.805399,0.367713,0.361742
PcaLimitWatchman(n_features=53),0.95817,0.248921,0.257048
IsoForestWatchman(n_features=53),0.044778,1.0,0.06892


SwatItrustDataset(E:\Datasets\SWaT\datasetA1)


,precision,recall,f1_score
ensemble(threshold=4),0.268927,0.856902,0.347944
ensemble(threshold=3),0.244791,0.905724,0.345886
ensemble(threshold=5),0.311374,0.804714,0.341294
DirectLimitWatchman(n_features=51),0.222668,0.96633,0.325686
DeepPredictWatchman(n_features=51),0.206669,0.905724,0.309418
IsoForestWatchman(n_features=51),0.168094,1.0,0.267875
LinearPredictWatchman(n_features=51),1.0,0.173822,0.213374
PcaLimitWatchman(n_features=51),0.577381,0.097252,0.161099
